<a href="https://colab.research.google.com/github/dmarx/notebooks/blob/text2video-modelscope/video_repaint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers huggingface_hub accelerate
!pip install git+https://github.com/huggingface/diffusers/ # need latest, currently unreleased 

In [3]:
from diffusers import RePaintPipeline, RePaintScheduler
import torch


model_repo = "damo-vilab/text-to-video-ms-1.7b" 
scheduler_repo = "google/ddpm-ema-celebahq-256"

scheduler = RePaintScheduler.from_pretrained(scheduler_repo)
pipe = RePaintPipeline.from_pretrained(model_repo, scheduler=scheduler, torch_dtype=torch.float16, variant="fp16")
pipe = pipe.to("cuda")

#pipe = DiffusionPipeline.from_pretrained("damo-vilab/text-to-video-ms-1.7b", torch_dtype=torch.float16, variant="fp16")
#pipe = pipe.to("cuda")

# the fuck? why is it redownloading? fml. ok, still faster. fine.

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

In [4]:
!pip install pims av

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pims: filename=PIMS-0.6.1-py3-none-any.whl size=82634 sha256=5b41873f3e1d9ca1884c652a407eb68237949afe71846b4ea20665efc84f9e40
  Stored in directory: /root/.cache/pip/wheels/c8/51/78/e8155864f53ff0c9e01fe656f64b55712ba60fb2a451757505
Successfully built pims


In [10]:
import pims

path_to_video = '/content/video-input-2.mp4'
#images = pims.open(path_to_video)

import av

container = av.open(path_to_video)

for frame in container.decode(video=0):
    frame.to_image().save('frame-%04d.jpg' % frame.index)

In [18]:
#type(frame.to_image()) # PIL
#dir(frame)
#frame.to_ndarray().shape # h, w
#frame.to_ndarray().max() # yeah this is in [0, 255]
dir(frame.to_ndarray)

['__call__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__self__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__text_signature__']

In [ ]:
!pip install torchvideo

In [24]:
dir(torchvideo)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__']

In [26]:
import torchvideo

In [31]:
import torch
#import torchvideo
#import torchvideo.transforms#
import torchvision

#container = av.open(path_to_video)
#frames = [frame.to_image() for i, frame in enumerate(container.decode(video=0)) if i<64]

frames = torchvision.io.read_video(path_to_video, output_format='TCHW')


/usr/local/lib/python3.9/dist-packages/torchvision/io/video.py:162: UserWarning: The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.
  warnings.warn("The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.")


In [32]:
#frames.shape
frames[0].shape # torch.Size([97, 3, 576, 1024])

torch.Size([97, 3, 576, 1024])

In [35]:
#len(frames) # 3
#frames[1] # tensor([], size=(1, 0))
frames[2] # {'video_fps': 24.0}

{'video_fps': 24.0}